In [39]:
!ls /home/amigos/data/rsky/20190907

xffts.ndf


In [40]:
path = "/home/amigos/data/rsky/20190907_190937/xffts.ndf"

In [41]:
import n2df
import numpy
import matplotlib.pyplot as plt
import os

In [42]:
n = n2df.Read(path)

In [43]:
obs_mode = n.read_obs_mode()
data_list = []
for i in range(16):
    exec("data_list.append(n.read_onearray({}))".format(i))

100%|██████████| 24/24 [00:00<00:00, 12206.05it/s]


In [44]:
obs_mode = numpy.array(obs_mode)
sky = obs_mode == "SKY"
hot = obs_mode == "HOT"

In [45]:
def get_tsys(dhot, dsky, thot):
    y = dhot / dsky
    tsys = thot / (y - 1.)
    return tsys

In [46]:
tsys_list = []
sky_list = []
hot_list = []
for i in data_list:
    i = numpy.array(i)
    _sky = sum(i[sky])/len(i[sky])
    _hot = sum(i[hot])/len(i[hot])
    tsys_list.append(get_tsys(_hot, _sky, 300))
    hot_list.append(_hot)
    sky_list.append(_sky)

In [ ]:
fig = plt.figure(figsize=(16,16))
ax = [fig.add_subplot(4, 4, i+1) for i in range(16)]
x = numpy.linspace(0, 2000, 32768)#XFFTS bw = 0-2000MHz
    
# plot hot
for i, (_ax, _tsys) in enumerate(zip(ax, tsys_list)):
    _ax.plot(x, hot_list[i], "r-", label="hot")
    _ax.plot(x, sky_list[i], "b-", label="sky")
    _ax0 = _ax.twinx()
    _ax0.plot(x, tsys_list[i], "g.", label="Tsys", alpha=0.1)
    _ax.set_xlabel('frequency [MHz]')
    _ax.set_ylabel("count")
    _ax.set_yscale("log")
    _ax0.set_ylabel('Tsys [K]')
    _ax.set_title('IF : {}'.format(i+1))
    tsys_av = numpy.mean(_tsys)
    _ax.text(0.05, 0.05, 'Tsys = %.2f'%(tsys_av), transform=_ax0.transAxes)
    _ax0.set_ylim(0, 600)
    _ax.legend()
    _ax.grid()

In [ ]:
plt.tight_layout()
plt.savefig(os.path.join(os.path.dirname(path), "result_rsky.png"))
plt.show()